# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [98]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [99]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [100]:
from secrets import IEX_CLOUD_API_TOKEN
symbol = 'AAPl'
api_url= f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2135371496755,
 'week52high': 148.9,
 'week52low': 83.56,
 'week52highSplitAdjustOnly': 151.91,
 'week52lowSplitAdjustOnly': 82.21,
 'week52change': 0.54606767089237,
 'sharesOutstanding': 17263139174,
 'float': 0,
 'avg10Volume': 76932837,
 'avg30Volume': 83313364,
 'day200MovingAvg': 131.54,
 'day50MovingAvg': 133.16,
 'employees': 150656,
 'ttmEPS': 4.51,
 'ttmDividendRate': 0.8570606167385952,
 'dividendYield': 0.006818010921387735,
 'nextDividendDate': '',
 'exDividendDate': '2021-05-05',
 'nextEarningsDate': '2021-07-25',
 'peRatio': 28.36219895658396,
 'beta': 1.4568613773627297,
 'maxChangePercent': 47.37053906538986,
 'year5ChangePercent': 4.529000968946603,
 'year2ChangePercent': 1.9055433785261087,
 'year1ChangePercent': 0.5865866473714574,
 'ytdChangePercent': -0.0493528351438046,
 'month6ChangePercent': 0.033726837138314224,
 'month3ChangePercent': 0.05148243529768409,
 'month1ChangePercent': -0.014057532925232515,
 'day30ChangePe

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [101]:
data['year1ChangePercent']

0.5865866473714574

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [102]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])



my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [103]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index = my_columns),
            ignore_index=True
        )
final_dataframe            

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,142.60,0.540869,N/A
1,AAL,25.00,0.467223,N/A
2,AAP,194.32,0.379666,N/A
3,AAPL,130.58,0.588956,N/A
4,ABBV,117.31,0.272702,N/A
...,...,...,...,...
500,YUM,119.91,0.278708,N/A
501,ZBH,159.83,0.237367,N/A
502,ZBRA,530.05,1.018256,N/A
503,ZION,60.54,0.639246,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [104]:
final_dataframe.sort_values('One-Year Price Return',ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace =True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,179,FCX,42.65,3.1224,N/A
1,275,LB,68.18,2.816024,N/A
2,253,IVZ,29.52,2.273965,N/A
3,148,DVN,32.52,1.702085,N/A
4,410,SIVB,613.42,1.674099,N/A
5,208,GPS,33.20,1.640811,N/A
6,314,MOS,37.32,1.639778,N/A
7,441,TPR,43.93,1.605043,N/A
8,344,NUE,109.47,1.540727,N/A
9,174,F,16.59,1.484965,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [105]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [106]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

C:\Users\Kodinariya's\AppData\Local\Programs\Python\Python37\Lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,179,FCX,42.65,3.1224,468
1,275,LB,68.18,2.816024,293
2,253,IVZ,29.52,2.273965,677
3,148,DVN,32.52,1.702085,615
4,410,SIVB,613.42,1.674099,32
5,208,GPS,33.20,1.640811,602
6,314,MOS,37.32,1.639778,535
7,441,TPR,43.93,1.605043,455
8,344,NUE,109.47,1.540727,182
9,174,F,16.59,1.484965,1205


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [120]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
    
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
columns = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

        
for column in columns:
    hqm_dataframe[f'{column} Price Return'] = hqm_dataframe[f'{column} Price Return'].astype('float')

        
hqm_dataframe.dtypes

Ticker                            object
Price                            float64
Number of Shares to Buy           object
One-Year Price Return            float64
One-Year Return Percentile        object
Six-Month Price Return           float64
Six-Month Return Percentile       object
Three-Month Price Return         float64
Three-Month Return Percentile     object
One-Month Price Return           float64
One-Month Return Percentile       object
HQM Score                         object
dtype: object

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [122]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col ] = score(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row,change_col ])/100

# Print each percentile score to make sure it was calculated properly
#for time_period in time_periods:
#    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,139.40,N/A,0.538144,0.647525,0.211422,0.526733,0.204882,0.720792,0.055297,0.708911,N/A
1,AAL,24.40,N/A,0.453678,0.550495,0.488273,0.910891,0.150875,0.49505,0.139704,0.946535,N/A
2,AAP,202.38,N/A,0.377272,0.473267,0.272314,0.663366,0.167354,0.562376,-0.056300,0.09901,N/A
3,AAPL,128.95,N/A,0.581616,0.679208,0.033577,0.146535,0.051420,0.172277,-0.013915,0.283168,N/A
4,ABBV,113.81,N/A,0.282129,0.334653,0.073367,0.217822,0.077668,0.241584,-0.013579,0.289109,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,122.81,N/A,0.277644,0.322772,0.133369,0.376238,0.152565,0.50099,-0.000914,0.354455,N/A
501,ZBH,165.41,N/A,0.245757,0.283168,0.071012,0.213861,0.020151,0.126733,-0.080946,0.055446,N/A
502,ZBRA,518.05,N/A,0.992564,0.918812,0.374670,0.817822,0.080719,0.257426,0.102282,0.885149,N/A
503,ZION,60.33,N/A,0.638921,0.736634,0.351427,0.794059,0.084332,0.269307,0.019713,0.493069,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [123]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row,'HQM Score'] = mean(momentum_percentiles) 
hqm_dataframe

    

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,139.40,N/A,0.538144,0.647525,0.211422,0.526733,0.204882,0.720792,0.055297,0.708911,0.65099
1,AAL,24.40,N/A,0.453678,0.550495,0.488273,0.910891,0.150875,0.49505,0.139704,0.946535,0.725743
2,AAP,202.38,N/A,0.377272,0.473267,0.272314,0.663366,0.167354,0.562376,-0.056300,0.09901,0.449505
3,AAPL,128.95,N/A,0.581616,0.679208,0.033577,0.146535,0.051420,0.172277,-0.013915,0.283168,0.320297
4,ABBV,113.81,N/A,0.282129,0.334653,0.073367,0.217822,0.077668,0.241584,-0.013579,0.289109,0.270792
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,122.81,N/A,0.277644,0.322772,0.133369,0.376238,0.152565,0.50099,-0.000914,0.354455,0.388614
501,ZBH,165.41,N/A,0.245757,0.283168,0.071012,0.213861,0.020151,0.126733,-0.080946,0.055446,0.169802
502,ZBRA,518.05,N/A,0.992564,0.918812,0.374670,0.817822,0.080719,0.257426,0.102282,0.885149,0.719802
503,ZION,60.33,N/A,0.638921,0.736634,0.351427,0.794059,0.084332,0.269307,0.019713,0.493069,0.573267


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [124]:
hqm_dataframe.sort_values('HQM Score',ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index( drop = True,inplace =True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NUE,111.94,N/A,1.578707,0.976238,0.963904,0.990099,0.817434,0.992079,0.214076,0.974257,0.983168
1,DVN,32.51,N/A,1.673449,0.984158,0.937599,0.988119,0.366091,0.964356,0.352689,0.990099,0.981683
2,F,16.13,N/A,1.444788,0.972277,0.722163,0.978218,0.352917,0.962376,0.402057,0.992079,0.976238
3,DXC,39.96,N/A,1.324935,0.968317,0.601808,0.958416,0.548688,0.990099,0.224411,0.978218,0.973762
4,COF,172.49,N/A,1.189591,0.946535,0.829319,0.984158,0.324246,0.940594,0.100027,0.879208,0.937624
5,SYF,50.79,N/A,1.125143,0.942574,0.547518,0.942574,0.294868,0.916832,0.122271,0.918812,0.930198
6,AMAT,141.41,N/A,1.489453,0.974257,0.596864,0.954455,0.303198,0.922772,0.090453,0.855446,0.926733
7,SLB,37.34,N/A,0.834877,0.855446,0.564163,0.946535,0.283560,0.89901,0.287044,0.984158,0.921287
8,IRM,47.48,N/A,0.732103,0.80198,0.649327,0.964356,0.332071,0.946535,0.148144,0.956436,0.917327
9,BEN,37.33,N/A,0.735625,0.805941,0.564852,0.948515,0.419311,0.978218,0.117550,0.912871,0.911386


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [125]:
portfolio_input()

Enter the value of your portfolio:1000000


In [126]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

C:\Users\Kodinariya's\AppData\Local\Programs\Python\Python37\Lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NUE,111.94,178,1.578707,0.976238,0.963904,0.990099,0.817434,0.992079,0.214076,0.974257,0.983168
1,DVN,32.51,615,1.673449,0.984158,0.937599,0.988119,0.366091,0.964356,0.352689,0.990099,0.981683
2,F,16.13,1239,1.444788,0.972277,0.722163,0.978218,0.352917,0.962376,0.402057,0.992079,0.976238
3,DXC,39.96,500,1.324935,0.968317,0.601808,0.958416,0.548688,0.990099,0.224411,0.978218,0.973762
4,COF,172.49,115,1.189591,0.946535,0.829319,0.984158,0.324246,0.940594,0.100027,0.879208,0.937624
5,SYF,50.79,393,1.125143,0.942574,0.547518,0.942574,0.294868,0.916832,0.122271,0.918812,0.930198
6,AMAT,141.41,141,1.489453,0.974257,0.596864,0.954455,0.303198,0.922772,0.090453,0.855446,0.926733
7,SLB,37.34,535,0.834877,0.855446,0.564163,0.946535,0.283560,0.89901,0.287044,0.984158,0.921287
8,IRM,47.48,421,0.732103,0.80198,0.649327,0.964356,0.332071,0.946535,0.148144,0.956436,0.917327
9,BEN,37.33,535,0.735625,0.805941,0.564852,0.948515,0.419311,0.978218,0.117550,0.912871,0.911386


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [127]:
writer= pd.ExcelWriter('momentum_stretagy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [128]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [129]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [130]:
writer.save()